### 풍자 문장 생성

In [1]:
# 토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_generate_0526")

In [3]:
# llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=1.0, model_name="gpt-4o")

In [ ]:
# prompt 가져오기
from langchain_core.prompts import load_prompt

prompt = load_prompt(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/generate_sarcasm_zeroshot_jeonghoon.yaml",
    encoding="utf-8",
)
prompt

In [ ]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional


# ① 원하는 데이터 구조 정의
class SarcasmOutput(BaseModel):
    sarcasm: str = Field(description="주어진 context를 반어적이고 과장하여 표현한 문장")
    non_sarcasm: str = Field(description="주어진 context를 non_sarcasm으로 표현한 문장")


# ② 출력 파서 설정
output_parser = PydanticOutputParser(pydantic_object=SarcasmOutput)

# ③ 출력 포맷 지시사항 확인
print(output_parser.get_format_instructions())

In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
# 체인 생성
chain = prompt | llm | output_parser

In [ ]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/negative_data_0531.csv",
    encoding="utf-8",
)

# df = df[300:639].copy()
df.info()

In [10]:
# input 데이터 로드
input_data = df["context"]

input_data_list = input_data.to_list()

In [ ]:
results = []

for context in input_data_list:
    try:
        # LLM 호출 및 결과 반환
        result = chain.invoke(
            {"context": context, "format_instructions": format_instructions}
        )

        # 결과가 dict 또는 SarcasmOutput 객체라면 속성 접근
        sarcasm = getattr(result, "sarcasm", None)
        non_sarcasm = getattr(result, "non_sarcasm", None)

    except Exception as e:
        print(f"[ERROR] context 처리 중 오류 발생:\n{context}\n{e}")
        sarcasm = None
        non_sarcasm = None

    results.append(
        {
            "context": context,
            "sarcasm": sarcasm,
            "non_sarcasm": non_sarcasm,
            "explanation": None,
        }
    )

# DataFrame 생성 및 저장
test_df = pd.DataFrame(results)

test_df.head()

test_df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_0531.csv",
    encoding="utf-8-sig",
)


In [ ]:
import pandas as pd

test_df = pd.DataFrame(results)

test_df.info()

In [ ]:
test_df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/1000개_temperature_1.0_zeroshot_1.csv",
    encoding="utf-8-sig",
)

### 칼럼명 통일

In [ ]:
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_0531.csv",
    encoding="utf-8-sig",
)

df.info()

In [ ]:
# sarcasm 데이터 가공
df_sarcasm = df[["context", "sarcasm"]].copy()
df_sarcasm.rename(columns={"sarcasm": "response"}, inplace=True)
df_sarcasm["label"] = "Sarcasm"
df_sarcasm["sarcasm_explanation"] = None

# non-sarcasm 데이터 가공
df_non_sarcasm = df[["context", "non_sarcasm"]].copy()
df_non_sarcasm.rename(columns={"non_sarcasm": "response"}, inplace=True)
df_non_sarcasm["label"] = "Non-sarcasm"
df_non_sarcasm["sarcasm_explanation"] = None

# 병합 후 인덱스 재설정
df_final = pd.concat([df_sarcasm, df_non_sarcasm], ignore_index=True)
df_final.reset_index(inplace=True)

# 결과 출력 예시
df_final.info()

In [14]:
# 결과 저장 (선택)
df_final.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_전처리_0531.csv",
    index=False,
)

### sarcasm, non_sarcasm explanation 작성

In [ ]:
# 토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_generate_0526")

In [17]:
# llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=1.0, model_name="gpt-4o-mini")

In [ ]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_전처리_0531.csv",
    encoding="utf-8",
)

df = df.dropna(subset=['context'])
df.info()


In [19]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "response", "label"],
    template="""
당신은 sarcasm과 non_sarcasm을 구분하는 언어 전문가입니다.
다음 내용을 참고하여, response 문장이 '{label}'인 이유를 한 문장으로 작성해주세요.

context: {context}
response: {response}
label: {label}

#Answer:
""",
)

In [ ]:
from tqdm import tqdm

explanations = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    input_data = {
        "context": row["context"],
        "response": row["response"],
        "label": row["label"],
    }
    answer = llm.invoke(prompt.format(**input_data)).content.strip()
    explanations.append(answer)

df["sarcasm_explanation"] = explanations

In [ ]:
df.head()

In [22]:
df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/0531_최종.csv"
)